In [95]:
import pandas as pd
import tensorflow as tf
import sklearn
import scikeras

from scikeras.wrappers import KerasRegressor
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import utils as np_utils
from tensorflow.keras import backend as k
from tensorflow.keras.models import Sequential
from sklearn.model_selection import cross_val_score

In [96]:
base = pd.read_csv('autos.csv', encoding = 'ISO-8859-1')
base = base.drop('dateCrawled', axis = 1)
base = base.drop('dateCreated', axis = 1)
base = base.drop('nrOfPictures', axis = 1)
base = base.drop('postalCode', axis = 1)
base = base.drop('lastSeen', axis = 1)
base = base.drop('name', axis = 1)
base = base.drop('seller', axis = 1)
base = base.drop('offerType', axis = 1)

In [97]:
valores = {'vehicleType': 'limousine', 'gearbox': 'manuell', 
           'model': 'golf', 'fuelType': 'benzin', 
           'notRepairedDamage': 'nein'}

In [98]:
base = base.fillna(value = valores)

In [99]:
previsores = base.iloc[:, 1:13].values
preco_real = base.iloc[:, 0].values

In [100]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
labelencoder_previsores = LabelEncoder()
previsores[:, 0] = labelencoder_previsores.fit_transform(previsores[:, 0])
previsores[:, 1] = labelencoder_previsores.fit_transform(previsores[:, 1])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 5] = labelencoder_previsores.fit_transform(previsores[:, 5])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_previsores.fit_transform(previsores[:, 9])
previsores[:, 10] = labelencoder_previsores.fit_transform(previsores[:, 10])

In [101]:
categorical_columns = [0, 1, 3, 5, 8, 9, 10]
onehotencoder = ct = ColumnTransformer(
    [('one_hot_encoder', OneHotEncoder(), categorical_columns)],
    remainder='passthrough')
data_transformed = ct.fit_transform(previsores).toarray()

In [102]:
def criarRede():
    k.clear_session()
    regressor = Sequential([
    tf.keras.layers.InputLayer(shape=(11,)),
    tf.keras.layers.Dense(units=158, activation='relu'),
    tf.keras.layers.Dense(units=158, activation='relu'),
    tf.keras.layers.Dense(units=1, activation='linear')])
    regressor.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mean_absolute_error'])
    return regressor

In [103]:
regressor = KerasRegressor(model= criarRede, epochs = 100, batch_size = 300)

In [104]:
resultados = cross_val_score(estimator=regressor, X=previsores, y=preco_real, cv=10, scoring='neg_mean_absolute_error')

Epoch 1/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 12138.8486 - mean_absolute_error: 12138.8486
Epoch 2/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 11553.9336 - mean_absolute_error: 11553.9336
Epoch 3/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 15005.6670 - mean_absolute_error: 15005.6670
Epoch 4/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 17724.5703 - mean_absolute_error: 17724.5703
Epoch 5/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 20134.8301 - mean_absolute_error: 20134.8301
Epoch 6/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 25032.8184 - mean_absolute_error: 25032.8184
Epoch 7/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 36877.0938 - mean_absolute_error: 36877.0938
Epoch 8/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 24327.7598 - mean_absolute_error: 24327.7598
Epoch 9/100
1115/1115 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 10071.5479 - mean_absolute_error: 10071.5479
Epoch 10/100
1115/1115 ━━━━━